# Regridding Data

In [ ]:
cd ..

In [ ]:
from omegaconf import OmegaConf
import hydra
import xarray as xr

import oceanbench._src.geoprocessing.validation as geoval

In [ ]:
import importlib
importlib.reload(geoval)

In [ ]:
raw_natl = xr.open_dataset('../sla-data-registry/NATL60/NATL/ref_new/NATL60-CJM165_NATL_ssh_y2013.1y.nc')

In [ ]:
raw_natl

## Preprocessing steps
- set time units "seconds since 2012-10-01"
- decode times to datetime
- add longitude and latitude units
- add sea surface height unit
- select domain

**Decode time**

In [ ]:
print('Before: ', raw_natl.time)
print()
natl = geoval.decode_cf_time(raw_natl, units="seconds since 2012-10-01")
print('After: ', natl.time)
                      

**Validate lat lon coordinates**

In [ ]:
print('Before: ', natl.lon.attrs, natl.lat.attrs)
print()
natl = geoval.validate_latlon(natl)
print('After: ', natl.lon.attrs, natl.lat.attrs)

**Validate ssh variable**

In [ ]:
print('Before: ', natl.ssh.attrs)
print()
natl = geoval.validate_ssh(natl)
print('After: ', natl.ssh.attrs)


In [ ]:
print('Before: ', natl.dims)
print()
final_natl = natl.sel(lat=slice(32, 44), lon=slice(-66, -54), time=slice('2013-01-10', '2013-03-10'))
print('After: ', final_natl.dims)

In [ ]:
final_natl

## Using configuration for processing

In [ ]:
import yaml
from IPython.display import Markdown, display

def disp_config(cfg):
    display(Markdown("""```yaml\n""" +yaml.dump(OmegaConf.to_container(cfg), default_flow_style=None, indent=2)+"""\n```"""))

In [ ]:
data_cfg = OmegaConf.load('config/data/gridded.yaml')
data = hydra.utils.call(data_cfg)
disp_config(data_cfg)

In [ ]:
key = 'natl'
OmegaConf.resolve(data_cfg[key])
disp_config(data_cfg[key])
data[key]()

In [ ]:
key = 'oi'
OmegaConf.resolve(data_cfg[key])
disp_config(data_cfg[key])
data[key]()

In [ ]:
key = 'obs'
OmegaConf.resolve(data_cfg[key])
disp_config(data_cfg[key])
data[key]()

In [ ]:
hydra.utils.call(data).natl

In [ ]:
hydra.utils.call(data_cfg).oi()